In [1]:
%%time
import pandas as pd
pd.set_option('display.max_columns', None)
df = pd.read_csv('/Volumes/transcend/大檔案/中華郵政/雙北市ACC.csv')
df_post = pd.read_csv('/Volumes/transcend/大檔案/中華郵政/郵務局號.csv')
df_post = df_post[['郵務局號','局名']]
df_post = df_post.rename(columns={'郵務局號': 'ACC01'})
df_post['局名'] = df_post['局名'].str.replace('郵局','')
df = pd.merge(df, df_post, on='ACC01')

<string>:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 2min 2s, sys: 45.8 s, total: 2min 48s
Wall time: 2min 58s


In [5]:
df.head(1)

,ACC01,ACC02,ACC03,ACC04,ACC05,ACC06,ACC07,ACC10,ACC11,ACC12,ACC13,ACC14,ACC15,ACC16,ACC17,ACC18,ACC19,ACC20,ACC21,ACC22,ACC23,ACC24,ACC25,ACC26,ACC27,ACC28,ACC29,ACC30,ACC31,ACC32,ACC33,ACC34,ACC35,ACC36,ACC37,ACC38,ACC39,ACC40,局名
0,100117,0,1,1,0.0,1070102,75412,100999,NaN,0,1,1,1,4.0,0,0,0,2,0,NaN,0,1,0,0,0,2.0,普通掛號公件信函,NaN,1.0,1,2000,1,90021210011701,90021210011701,NaN,NaN,10701,1,臺北大安


- ACC31 (計費方式1重量計費2尺寸計費3單一4上收5首次使用便利箱袋6重複使用便利箱袋)

In [7]:
df['ACC31'].value_counts()

1.0    8844088
3.0     681558
5.0     507245
6.0      71918
4.0         86
Name: ACC31, dtype: int64

- ACC14 (基本郵件種類(1函件2包裹3快捷))
- ACC15
<br>函件:1.信函 2.小包 3.雜誌 4.新聞紙 5.印刷物 6.明信片 7.郵簡 8.盲人文件 9.印刷物專袋
<br>包裹:1.一般包裹 2.輕笨包裹 3.勞軍包裹4.特產快遞
<br>快捷:1.一般快捷 2.文件快捷 3.商品快捷 4.冷凍優鮮配 5.冷藏優鮮配 6.特產快遞"
<br><br>
- ACC18 (0無1普通回執2掛號回執3限時掛號回執)
- ACC24 (是否掛號(0否1是))
- ACC25 (速度別0普通1限時快遞)
- ACC32 (本頁件數(該頁碼總件數))
- ACC33 (本頁重量(公斤)(該頁碼總重量))

### 方法一

In [140]:
from tqdm import tqdm_notebook
price, error_index = [], []
for i in tqdm_notebook(range(len(df))):
    # 只考慮國內、重量計費
    if (df['ACC27'][i] == 0) & (df['ACC31'][i]==1):
        # 信函
        if (df['ACC14'][i] == 1) & (df['ACC15'][i]==1):
            # 普通
            if (df['ACC18'][i] == 0) & (df['ACC24'][i] == 0) & (df['ACC25'][i] == 0):
                if df['ACC33'][i] <= 20:
                    price.append(8)
                elif 20 < df['ACC33'][i] <= 50:
                    price.append(16)
                elif 50 < df['ACC33'][i] <= 100:
                    price.append(24)
                elif 100 < df['ACC33'][i] <= 250:
                    price.append(40)
                elif 250 < df['ACC33'][i] <= 500:
                    price.append(72)
                elif 500 < df['ACC33'][i] <= 1000:
                    price.append(112)
                elif 1000 < df['ACC33'][i] <= 2000:
                    price.append(160)
                else:
                    price.append(None)
                    error_index.append(i)
            # 限時
            elif (df['ACC18'][i] == 0) & (df['ACC24'][i] == 0) & (df['ACC25'][i] == 1):
                if df['ACC33'][i] <= 20:
                    price.append(15)
                elif 20 < df['ACC33'][i] <= 50:
                    price.append(23)
                elif 50 < df['ACC33'][i] <= 100:
                    price.append(31)
                elif 100 < df['ACC33'][i] <= 250:
                    price.append(47)
                elif 250 < df['ACC33'][i] <= 500:
                    price.append(79)
                elif 500 < df['ACC33'][i] <= 1000:
                    price.append(119)
                elif 1000 < df['ACC33'][i] <= 2000:
                    price.append(167)
                else:
                    price.append(None)
                    error_index.append(i)
            # 普通掛號
            elif (df['ACC18'][i] == 0) & (df['ACC24'][i] == 1) & (df['ACC25'][i] == 0):
                if df['ACC33'][i] <= 20:
                    price.append(28)
                elif 20 < df['ACC33'][i] <= 50:
                    price.append(36)
                elif 50 < df['ACC33'][i] <= 100:
                    price.append(44)
                elif 100 < df['ACC33'][i] <= 250:
                    price.append(60)
                elif 250 < df['ACC33'][i] <= 500:
                    price.append(92)
                elif 500 < df['ACC33'][i] <= 1000:
                    price.append(132)
                elif 1000 < df['ACC33'][i] <= 2000:
                    price.append(180)
                else:
                    price.append(None)
                    error_index.append(i)
            # 限時掛號
            elif (df['ACC18'][i] == 0) & (df['ACC24'][i] == 1) & (df['ACC25'][i] == 1):
                if df['ACC33'][i] <= 20:
                    price.append(35)
                elif 20 < df['ACC33'][i] <= 50:
                    price.append(43)
                elif 50 < df['ACC33'][i] <= 100:
                    price.append(51)
                elif 100 < df['ACC33'][i] <= 250:
                    price.append(67)
                elif 250 < df['ACC33'][i] <= 500:
                    price.append(99)
                elif 500 < df['ACC33'][i] <= 1000:
                    price.append(139)
                elif 1000 < df['ACC33'][i] <= 2000:
                    price.append(187)
                else:
                    price.append(None)
                    error_index.append(i)
            # 普通掛號附回執(回執為平信)
            elif (df['ACC18'][i] == 1) & (df['ACC24'][i] == 1) & (df['ACC25'][i] == 0):
                if df['ACC33'][i] <= 20:
                    price.append(43)
                elif 20 < df['ACC33'][i] <= 50:
                    price.append(51)
                elif 50 < df['ACC33'][i] <= 100:
                    price.append(59)
                elif 100 < df['ACC33'][i] <= 250:
                    price.append(75)
                elif 250 < df['ACC33'][i] <= 500:
                    price.append(107)
                elif 500 < df['ACC33'][i] <= 1000:
                    price.append(147)
                elif 1000 < df['ACC33'][i] <= 2000:
                    price.append(195)
                else:
                    price.append(None)
                    error_index.append(i)
            # 限時掛號附回執(回執為平信)
            elif (df['ACC18'][i] == 2) & (df['ACC24'][i] == 1) & (df['ACC25'][i] == 0):
                if df['ACC33'][i] <= 20:
                    price.append(50)
                elif 20 < df['ACC33'][i] <= 50:
                    price.append(58)
                elif 50 < df['ACC33'][i] <= 100:
                    price.append(66)
                elif 100 < df['ACC33'][i] <= 250:
                    price.append(82)
                elif 250 < df['ACC33'][i] <= 500:
                    price.append(114)
                elif 500 < df['ACC33'][i] <= 1000:
                    price.append(154)
                elif 1000 < df['ACC33'][i] <= 2000:
                    price.append(202)
                else:
                    price.append(None)
                    error_index.append(i)
            else:
                price.append(None)
                error_index.append(i)
        # 印刷物
        elif (df['ACC14'][i] == 1) & (df['ACC15'][i]==5):
            # 普通
            if (df['ACC18'][i] == 0) & (df['ACC24'][i] == 0) & (df['ACC25'][i] == 0):
                if df['ACC33'][i] <= 50:
                    price.append(6)
                elif 50 < df['ACC33'][i] <= 100:
                    price.append(11)
                elif 100 < df['ACC33'][i] <= 250:
                    price.append(16)
                elif 250 < df['ACC33'][i] <= 500:
                    price.append(32)
                elif 500 < df['ACC33'][i] <= 1000:
                    price.append(56)
                elif 1000 < df['ACC33'][i] <= 2000:
                    price.append(88)
                else:
                    price.append(None)
                    error_index.append(i)
            # 限時
            elif (df['ACC18'][i] == 0) & (df['ACC24'][i] == 0) & (df['ACC25'][i] == 1):
                if df['ACC33'][i] <= 50:
                    price.append(13)
                elif 50 < df['ACC33'][i] <= 100:
                    price.append(18)
                elif 100 < df['ACC33'][i] <= 250:
                    price.append(23)
                elif 250 < df['ACC33'][i] <= 500:
                    price.append(39)
                elif 500 < df['ACC33'][i] <= 1000:
                    price.append(63)
                elif 1000 < df['ACC33'][i] <= 2000:
                    price.append(95)
                else:
                    price.append(None)
                    error_index.append(i)
            # 普通掛號
            elif (df['ACC18'][i] == 0) & (df['ACC24'][i] == 1) & (df['ACC25'][i] == 0):
                if df['ACC33'][i] <= 50:
                    price.append(26)
                elif 50 < df['ACC33'][i] <= 100:
                    price.append(31)
                elif 100 < df['ACC33'][i] <= 250:
                    price.append(36)
                elif 250 < df['ACC33'][i] <= 500:
                    price.append(52)
                elif 500 < df['ACC33'][i] <= 1000:
                    price.append(76)
                elif 1000 < df['ACC33'][i] <= 2000:
                    price.append(108)
                else:
                    price.append(None)
                    error_index.append(i)
            # 限時掛號
            elif (df['ACC18'][i] == 0) & (df['ACC24'][i] == 1) & (df['ACC25'][i] == 1):
                if df['ACC33'][i] <= 50:
                    price.append(33)
                elif 50 < df['ACC33'][i] <= 100:
                    price.append(38)
                elif 100 < df['ACC33'][i] <= 250:
                    price.append(43)
                elif 250 < df['ACC33'][i] <= 500:
                    price.append(59)
                elif 500 < df['ACC33'][i] <= 1000:
                    price.append(83)
                elif 1000 < df['ACC33'][i] <= 2000:
                    price.append(115)
                else:
                    price.append(None)
                    error_index.append(i)
            # 普通掛號附回執(回執為平信)
            elif (df['ACC18'][i] == 1) & (df['ACC24'][i] == 1) & (df['ACC25'][i] == 0):
                if df['ACC33'][i] <= 50:
                    price.append(41)
                elif 50 < df['ACC33'][i] <= 100:
                    price.append(46)
                elif 100 < df['ACC33'][i] <= 250:
                    price.append(51)
                elif 250 < df['ACC33'][i] <= 500:
                    price.append(67)
                elif 500 < df['ACC33'][i] <= 1000:
                    price.append(91)
                elif 1000 < df['ACC33'][i] <= 2000:
                    price.append(123)
                else:
                    price.append(None)
                    error_index.append(i)
            # 限時掛號附回執(回執為平信)，過少
            else:
                price.append(None)
                error_index.append(i)
        # 小包
        elif (df['ACC14'][i] == 1) & (df['ACC15'][i]==2):
            # 普通
            if (df['ACC18'][i] == 0) & (df['ACC24'][i] == 0) & (df['ACC25'][i] == 0):
                if df['ACC33'][i] <= 100:
                    price.append(12)
                elif 100 < df['ACC33'][i] <= 200:
                    price.append(24)
                elif 200 < df['ACC33'][i] <= 300:
                    price.append(36)
                elif 300 < df['ACC33'][i] <= 400:
                    price.append(48)
                elif 400 < df['ACC33'][i] <= 500:
                    price.append(60)
                elif 500 < df['ACC33'][i] <= 600:
                    price.append(72)
                elif 600 < df['ACC33'][i] <= 700:
                    price.append(84)
                elif 700 < df['ACC33'][i] <= 800:
                    price.append(96)
                elif 800 < df['ACC33'][i] <= 900:
                    price.append(108)
                elif 900 < df['ACC33'][i] <= 1000:
                    price.append(120)
                else:
                    price.append(None)
                    error_index.append(i)
            # 限時
            elif (df['ACC18'][i] == 0) & (df['ACC24'][i] == 0) & (df['ACC25'][i] == 1):
                if df['ACC33'][i] <= 100:
                    price.append(19)
                elif 100 < df['ACC33'][i] <= 200:
                    price.append(31)
                elif 200 < df['ACC33'][i] <= 300:
                    price.append(43)
                elif 300 < df['ACC33'][i] <= 400:
                    price.append(55)
                elif 400 < df['ACC33'][i] <= 500:
                    price.append(67)
                elif 500 < df['ACC33'][i] <= 600:
                    price.append(79)
                elif 600 < df['ACC33'][i] <= 700:
                    price.append(91)
                elif 700 < df['ACC33'][i] <= 800:
                    price.append(103)
                elif 800 < df['ACC33'][i] <= 900:
                    price.append(115)
                elif 900 < df['ACC33'][i] <= 1000:
                    price.append(127)
                else:
                    price.append(None)
                    error_index.append(i)
            # 普通掛號
            elif (df['ACC18'][i] == 0) & (df['ACC24'][i] == 1) & (df['ACC25'][i] == 0):
                if df['ACC33'][i] <= 100:
                    price.append(32)
                elif 100 < df['ACC33'][i] <= 200:
                    price.append(44)
                elif 200 < df['ACC33'][i] <= 300:
                    price.append(56)
                elif 300 < df['ACC33'][i] <= 400:
                    price.append(68)
                elif 400 < df['ACC33'][i] <= 500:
                    price.append(80)
                elif 500 < df['ACC33'][i] <= 600:
                    price.append(92)
                elif 600 < df['ACC33'][i] <= 700:
                    price.append(104)
                elif 700 < df['ACC33'][i] <= 800:
                    price.append(116)
                elif 800 < df['ACC33'][i] <= 900:
                    price.append(128)
                elif 900 < df['ACC33'][i] <= 1000:
                    price.append(140)
                else:
                    price.append(None)
                    error_index.append(i)
            # 限時掛號
            elif (df['ACC18'][i] == 0) & (df['ACC24'][i] == 1) & (df['ACC25'][i] == 1):
                if df['ACC33'][i] <= 100:
                    price.append(39)
                elif 100 < df['ACC33'][i] <= 200:
                    price.append(51)
                elif 200 < df['ACC33'][i] <= 300:
                    price.append(63)
                elif 300 < df['ACC33'][i] <= 400:
                    price.append(75)
                elif 400 < df['ACC33'][i] <= 500:
                    price.append(87)
                elif 500 < df['ACC33'][i] <= 600:
                    price.append(99)
                elif 600 < df['ACC33'][i] <= 700:
                    price.append(111)
                elif 700 < df['ACC33'][i] <= 800:
                    price.append(123)
                elif 800 < df['ACC33'][i] <= 900:
                    price.append(135)
                elif 900 < df['ACC33'][i] <= 1000:
                    price.append(147)
                else:
                    price.append(None)
                    error_index.append(i)
            # 普通掛號附回執(回執為平信)
            elif (df['ACC18'][i] == 1) & (df['ACC24'][i] == 1) & (df['ACC25'][i] == 0):
                if df['ACC33'][i] <= 100:
                    price.append(47)
                elif 100 < df['ACC33'][i] <= 200:
                    price.append(59)
                elif 200 < df['ACC33'][i] <= 300:
                    price.append(71)
                elif 300 < df['ACC33'][i] <= 400:
                    price.append(83)
                elif 400 < df['ACC33'][i] <= 500:
                    price.append(95)
                elif 500 < df['ACC33'][i] <= 600:
                    price.append(107)
                elif 600 < df['ACC33'][i] <= 700:
                    price.append(119)
                elif 700 < df['ACC33'][i] <= 800:
                    price.append(131)
                elif 800 < df['ACC33'][i] <= 900:
                    price.append(143)
                elif 900 < df['ACC33'][i] <= 1000:
                    price.append(155)
                else:
                    price.append(None)
                    error_index.append(i)
            # 限時掛號附回執(回執為平信)，過少
            else:
                price.append(None)
                error_index.append(i)
        # 新聞紙
        elif (df['ACC14'][i] == 1) & (df['ACC15'][i]==4):
            price.append(((int(df['ACC33'][i])/50)+1)*2)
        # 雜誌
        elif (df['ACC14'][i] == 1) & (df['ACC15'][i]==3):
            price.append(((int(df['ACC33'][i])/50)+1)*3)
        # 明信片
        elif (df['ACC14'][i] == 1) & (df['ACC15'][i]==6):
            if df['ACC25'][i]==0:
                price.append(df['ACC32'][i]*5)
            elif df['ACC25'][i]==1:
                price.append(df['ACC32'][i]*12)
            else:
                price.append(None)
                error_index.append(i)
        # 郵簡
        elif (df['ACC14'][i] == 1) & (df['ACC15'][i]==7):
            price.append(df['ACC32'][i]*6)

        # 包裹
        elif df['ACC14'][i] == 2:
            if (df['ACC29'][i] == '同縣市包裹') | (df['ACC29'][i] == '國內同縣市包裹'):
                if df['ACC33'][i] <= 5000:
                    price.append(70)
                elif 5000 < df['ACC33'][i] <= 10000:
                    price.append(90)
                elif 10000 < df['ACC33'][i] <= 15000:
                    price.append(110)
                elif 15000 < df['ACC33'][i] <= 20000:
                    price.append(135)
                else:
                    price.append(None)
                    error_index.append(i)
            elif (df['ACC29'][i] == '不同縣市包裹') | (df['ACC29'][i] == '代收貨價不同縣市包裹') | (df['ACC29'][i] == '國內不同縣市包裹'):
                if df['ACC33'][i] <= 5000:
                    price.append(80)
                elif 5000 < df['ACC33'][i] <= 10000:
                    price.append(100)
                elif 10000 < df['ACC33'][i] <= 15000:
                    price.append(120)
                elif 15000 < df['ACC33'][i] <= 20000:
                    price.append(145)
                else:
                    price.append(None)
                    error_index.append(i)
            elif df['ACC29'][i] == '島嶼間互寄包裹':
                if df['ACC33'][i] <= 5000:
                    price.append(100)
                elif 5000 < df['ACC33'][i] <= 10000:
                    price.append(125)
                elif 10000 < df['ACC33'][i] <= 15000:
                    price.append(150)
                elif 15000 < df['ACC33'][i] <= 20000:
                    price.append(180)
                else:
                    price.append(None)
                    error_index.append(i)
            else:
                price.append(None)
                error_index.append(i)

        # 快捷
        elif df['ACC14'][i] == 3:
            if (df['ACC29'][i] == '商品快捷') | (df['ACC29'][i] == '同縣市快捷') | (df['ACC29'][i] == '文件快捷') | (df['ACC29'][i] == '國內同縣市快捷'):
                if df['ACC33'][i] <= 250:
                    price.append(80)
                elif 250 < df['ACC33'][i] <= 1000:
                    price.append(100)
                elif 1000 < df['ACC33'][i] <= 5000:
                    price.append(130)
                elif 5000 < df['ACC33'][i] <= 10000:
                    price.append(170)
                elif 10000 < df['ACC33'][i] <= 15000:
                    price.append(210)
                elif 15000 < df['ACC33'][i] <= 20000:
                    price.append(250)
                else:
                    price.append(None)
                    error_index.append(i)
            elif (df['ACC29'][i] == '不同縣市快捷') | (df['ACC29'][i] == '國內不同縣市快捷'):
                if df['ACC33'][i] <= 250:
                    price.append(120)
                elif 250 < df['ACC33'][i] <= 1000:
                    price.append(150)
                elif 1000 < df['ACC33'][i] <= 5000:
                    price.append(190)
                elif 5000 < df['ACC33'][i] <= 10000:
                    price.append(250)
                elif 10000 < df['ACC33'][i] <= 15000:
                    price.append(320)
                elif 15000 < df['ACC33'][i] <= 20000:
                    price.append(400)
                else:
                    price.append(None)
                    error_index.append(i)
            elif df['ACC29'][i] == '島嶼間互寄快捷':
                if df['ACC33'][i] <= 250:
                    price.append(140)
                elif 250 < df['ACC33'][i] <= 1000:
                    price.append(170)
                elif 1000 < df['ACC33'][i] <= 5000:
                    price.append(200)
                elif 5000 < df['ACC33'][i] <= 10000:
                    price.append(275)
                elif 10000 < df['ACC33'][i] <= 15000:
                    price.append(350)
                elif 15000 < df['ACC33'][i] <= 20000:
                    price.append(425)
                else:
                    price.append(None)
                    error_index.append(i)
            else:
                price.append(None)
                error_index.append(i)
        else:
            price.append(None)
            error_index.append(i)
    else:
        price.append(None)
        error_index.append(i)

In [141]:
print(len(price))
print(len(error_index))

10104895
3814452


### 方法二

In [149]:
df['ACC29'].value_counts()

普通掛號信函             4243403
普通掛號小包              887589
不同縣市包裹              821913
國內普通掛號信函            612561
限時掛號信函              524414
同縣市包裹               490010
普通掛號公件信函            180593
國內普通信函              148536
普通掛號印刷物             130120
國內限時掛號信函            129523
國內普通掛號小包            125586
普通掛號信函附回執           112629
普通代收貨價小包            102667
商品快捷                101683
國內普通印刷物              78808
航空普通掛號小包             76897
國內同縣市包裹              73295
代收貨價不同縣市包裹           69372
不同縣市快捷               69168
同縣市快捷                69065
國內普通回執信函             67001
限時掛號小包               64396
航空包裹                 62228
國內不同縣市包裹             55921
航空普通掛號信函             50610
國內限時信函               48540
島嶼間互寄包裹              42142
文件快捷                 41535
限時掛號公件信函             32749
國內普通掛號信函附回執          31423
                    ...   
航空包裹附回執                  2
美加航空快遞信函                 2
限時掛號新聞紙                  2
普通非法定紙幣報值印刷物             2
國內限時掛號新聞紙                2
航空快遞保價信函                 2
港

In [151]:
from tqdm import tqdm_notebook
price, error_index = [], []
for i in tqdm_notebook(range(len(df))):
    if (df['ACC29'][i] == '普通掛號信函') | (df['ACC29'][i] == '國內普通掛號信函') | (df['ACC29'][i] == '普通掛號公件信函'):
        if df['ACC33'][i] <= 20:
            price.append(28)
        elif 20 < df['ACC33'][i] <= 50:
            price.append(36)
        elif 50 < df['ACC33'][i] <= 100:
            price.append(44)
        elif 100 < df['ACC33'][i] <= 250:
            price.append(60)
        elif 250 < df['ACC33'][i] <= 500:
            price.append(92)
        elif 500 < df['ACC33'][i] <= 1000:
            price.append(132)
        elif 1000 < df['ACC33'][i] <= 2000:
            price.append(180)
        else:
            price.append(None)
            error_index.append(i)
    elif df['ACC29'][i] == '普通掛號小包':
        if df['ACC33'][i] <= 100:
            price.append(32)
        elif 100 < df['ACC33'][i] <= 200:
            price.append(44)
        elif 200 < df['ACC33'][i] <= 300:
            price.append(56)
        elif 300 < df['ACC33'][i] <= 400:
            price.append(68)
        elif 400 < df['ACC33'][i] <= 500:
            price.append(80)
        elif 500 < df['ACC33'][i] <= 600:
            price.append(92)
        elif 600 < df['ACC33'][i] <= 700:
            price.append(104)
        elif 700 < df['ACC33'][i] <= 800:
            price.append(116)
        elif 800 < df['ACC33'][i] <= 900:
            price.append(128)
        elif 900 < df['ACC33'][i] <= 1000:
            price.append(140)
        else:
            price.append(None)
            error_index.append(i)
    elif (df['ACC29'][i] == '不同縣市包裹') | (df['ACC29'][i] == '代收貨價不同縣市包裹') | (df['ACC29'][i] == '國內不同縣市包裹'):
        if df['ACC33'][i] <= 5000:
            price.append(80)
        elif 5000 < df['ACC33'][i] <= 10000:
            price.append(100)
        elif 10000 < df['ACC33'][i] <= 15000:
            price.append(120)
        elif 15000 < df['ACC33'][i] <= 20000:
            price.append(145)
        else:
            price.append(None)
            error_index.append(i)
    elif df['ACC29'][i] == '限時掛號信函':
        if df['ACC33'][i] <= 20:
            price.append(35)
        elif 20 < df['ACC33'][i] <= 50:
            price.append(43)
        elif 50 < df['ACC33'][i] <= 100:
            price.append(51)
        elif 100 < df['ACC33'][i] <= 250:
            price.append(67)
        elif 250 < df['ACC33'][i] <= 500:
            price.append(99)
        elif 500 < df['ACC33'][i] <= 1000:
            price.append(139)
        elif 1000 < df['ACC33'][i] <= 2000:
            price.append(187)
        else:
            price.append(None)
            error_index.append(i)
    elif (df['ACC29'][i] == '同縣市包裹') | (df['ACC29'][i] == '國內同縣市包裹'):
        if df['ACC33'][i] <= 5000:
            price.append(70)
        elif 5000 < df['ACC33'][i] <= 10000:
            price.append(90)
        elif 10000 < df['ACC33'][i] <= 15000:
            price.append(110)
        elif 15000 < df['ACC33'][i] <= 20000:
            price.append(135)
        else:
            price.append(None)
            error_index.append(i)
    elif df['ACC29'][i] == '國內普通信函':
        if df['ACC33'][i] <= 20:
            price.append(8)
        elif 20 < df['ACC33'][i] <= 50:
            price.append(16)
        elif 50 < df['ACC33'][i] <= 100:
            price.append(24)
        elif 100 < df['ACC33'][i] <= 250:
            price.append(40)
        elif 250 < df['ACC33'][i] <= 500:
            price.append(72)
        elif 500 < df['ACC33'][i] <= 1000:
            price.append(112)
        elif 1000 < df['ACC33'][i] <= 2000:
            price.append(160)
        else:
            price.append(None)
            error_index.append(i)
    elif df['ACC29'][i] == '普通掛號印刷物':
        if df['ACC33'][i] <= 50:
            price.append(26)
        elif 50 < df['ACC33'][i] <= 100:
            price.append(31)
        elif 100 < df['ACC33'][i] <= 250:
            price.append(36)
        elif 250 < df['ACC33'][i] <= 500:
            price.append(52)
        elif 500 < df['ACC33'][i] <= 1000:
            price.append(76)
        elif 1000 < df['ACC33'][i] <= 2000:
            price.append(108)
        else:
            price.append(None)
            error_index.append(i)
    elif (df['ACC29'][i] == '國內限時掛號信函') | (df['ACC29'][i] == '限時掛號公件信函'):
        if df['ACC33'][i] <= 20:
            price.append(35)
        elif 20 < df['ACC33'][i] <= 50:
            price.append(43)
        elif 50 < df['ACC33'][i] <= 100:
            price.append(51)
        elif 100 < df['ACC33'][i] <= 250:
            price.append(67)
        elif 250 < df['ACC33'][i] <= 500:
            price.append(99)
        elif 500 < df['ACC33'][i] <= 1000:
            price.append(139)
        elif 1000 < df['ACC33'][i] <= 2000:
            price.append(187)
        else:
            price.append(None)
            error_index.append(i)
    elif df['ACC29'][i] == '國內普通掛號小包':
        if df['ACC33'][i] <= 100:
            price.append(32)
        elif 100 < df['ACC33'][i] <= 200:
            price.append(44)
        elif 200 < df['ACC33'][i] <= 300:
            price.append(56)
        elif 300 < df['ACC33'][i] <= 400:
            price.append(68)
        elif 400 < df['ACC33'][i] <= 500:
            price.append(80)
        elif 500 < df['ACC33'][i] <= 600:
            price.append(92)
        elif 600 < df['ACC33'][i] <= 700:
            price.append(104)
        elif 700 < df['ACC33'][i] <= 800:
            price.append(116)
        elif 800 < df['ACC33'][i] <= 900:
            price.append(128)
        elif 900 < df['ACC33'][i] <= 1000:
            price.append(140)
        else:
            price.append(None)
            error_index.append(i)
    elif (df['ACC29'][i] == '普通掛號信函附回執') | (df['ACC29'][i] == '國內普通掛號信函附回執'):
        if df['ACC33'][i] <= 20:
            price.append(43)
        elif 20 < df['ACC33'][i] <= 50:
            price.append(51)
        elif 50 < df['ACC33'][i] <= 100:
            price.append(59)
        elif 100 < df['ACC33'][i] <= 250:
            price.append(75)
        elif 250 < df['ACC33'][i] <= 500:
            price.append(107)
        elif 500 < df['ACC33'][i] <= 1000:
            price.append(147)
        elif 1000 < df['ACC33'][i] <= 2000:
            price.append(195)
        else:
            price.append(None)
            error_index.append(i)
    elif df['ACC29'][i] == '普通代收貨價小包':
        if df['ACC33'][i] <= 100:
            price.append(12)
        elif 100 < df['ACC33'][i] <= 200:
            price.append(24)
        elif 200 < df['ACC33'][i] <= 300:
            price.append(36)
        elif 300 < df['ACC33'][i] <= 400:
            price.append(48)
        elif 400 < df['ACC33'][i] <= 500:
            price.append(60)
        elif 500 < df['ACC33'][i] <= 600:
            price.append(72)
        elif 600 < df['ACC33'][i] <= 700:
            price.append(84)
        elif 700 < df['ACC33'][i] <= 800:
            price.append(96)
        elif 800 < df['ACC33'][i] <= 900:
            price.append(108)
        elif 900 < df['ACC33'][i] <= 1000:
            price.append(120)
        else:
            price.append(None)
            error_index.append(i)
    elif (df['ACC29'][i] == '商品快捷') | (df['ACC29'][i] == '同縣市快捷') | (df['ACC29'][i] == '文件快捷') | (df['ACC29'][i] == '國內同縣市快捷'):
        if df['ACC33'][i] <= 250:
            price.append(80)
        elif 250 < df['ACC33'][i] <= 1000:
            price.append(100)
        elif 1000 < df['ACC33'][i] <= 5000:
            price.append(130)
        elif 5000 < df['ACC33'][i] <= 10000:
            price.append(170)
        elif 10000 < df['ACC33'][i] <= 15000:
            price.append(210)
        elif 15000 < df['ACC33'][i] <= 20000:
            price.append(250)
        else:
            price.append(None)
            error_index.append(i)
    elif df['ACC29'][i] == '國內普通印刷物':
        if df['ACC33'][i] <= 50:
            price.append(6)
        elif 50 < df['ACC33'][i] <= 100:
            price.append(11)
        elif 100 < df['ACC33'][i] <= 250:
            price.append(16)
        elif 250 < df['ACC33'][i] <= 500:
            price.append(32)
        elif 500 < df['ACC33'][i] <= 1000:
            price.append(56)
        elif 1000 < df['ACC33'][i] <= 2000:
            price.append(88)
        else:
            price.append(None)
            error_index.append(i)
    elif (df['ACC29'][i] == '同縣市包裹') | (df['ACC29'][i] == '國內同縣市包裹'):
        if df['ACC33'][i] <= 5000:
            price.append(70)
        elif 5000 < df['ACC33'][i] <= 10000:
            price.append(90)
        elif 10000 < df['ACC33'][i] <= 15000:
            price.append(110)
        elif 15000 < df['ACC33'][i] <= 20000:
            price.append(135)
        else:
            price.append(None)
            error_index.append(i)
    elif (df['ACC29'][i] == '不同縣市包裹') | (df['ACC29'][i] == '代收貨價不同縣市包裹') | (df['ACC29'][i] == '國內不同縣市包裹'):
        if df['ACC33'][i] <= 5000:
            price.append(80)
        elif 5000 < df['ACC33'][i] <= 10000:
            price.append(100)
        elif 10000 < df['ACC33'][i] <= 15000:
            price.append(120)
        elif 15000 < df['ACC33'][i] <= 20000:
            price.append(145)
        else:
            price.append(None)
            error_index.append(i)
    elif (df['ACC29'][i] == '不同縣市快捷') | (df['ACC29'][i] == '國內不同縣市快捷'):
        if df['ACC33'][i] <= 250:
            price.append(120)
        elif 250 < df['ACC33'][i] <= 1000:
            price.append(150)
        elif 1000 < df['ACC33'][i] <= 5000:
            price.append(190)
        elif 5000 < df['ACC33'][i] <= 10000:
            price.append(250)
        elif 10000 < df['ACC33'][i] <= 15000:
            price.append(320)
        elif 15000 < df['ACC33'][i] <= 20000:
            price.append(400)
        else:
            price.append(None)
            error_index.append(i)
    elif df['ACC29'][i] == '限時掛號小包':
        if df['ACC33'][i] <= 100:
            price.append(39)
        elif 100 < df['ACC33'][i] <= 200:
            price.append(51)
        elif 200 < df['ACC33'][i] <= 300:
            price.append(63)
        elif 300 < df['ACC33'][i] <= 400:
            price.append(75)
        elif 400 < df['ACC33'][i] <= 500:
            price.append(87)
        elif 500 < df['ACC33'][i] <= 600:
            price.append(99)
        elif 600 < df['ACC33'][i] <= 700:
            price.append(111)
        elif 700 < df['ACC33'][i] <= 800:
            price.append(123)
        elif 800 < df['ACC33'][i] <= 900:
            price.append(135)
        elif 900 < df['ACC33'][i] <= 1000:
            price.append(147)
        else:
            price.append(None)
            error_index.append(i)
    elif df['ACC29'][i] == '國內限時信函':
        if df['ACC33'][i] <= 20:
            price.append(15)
        elif 20 < df['ACC33'][i] <= 50:
            price.append(23)
        elif 50 < df['ACC33'][i] <= 100:
            price.append(31)
        elif 100 < df['ACC33'][i] <= 250:
            price.append(47)
        elif 250 < df['ACC33'][i] <= 500:
            price.append(79)
        elif 500 < df['ACC33'][i] <= 1000:
            price.append(119)
        elif 1000 < df['ACC33'][i] <= 2000:
            price.append(167)
        else:
            price.append(None)
            error_index.append(i)
    elif df['ACC29'][i] == '島嶼間互寄包裹':
        if df['ACC33'][i] <= 5000:
            price.append(100)
        elif 5000 < df['ACC33'][i] <= 10000:
            price.append(125)
        elif 10000 < df['ACC33'][i] <= 15000:
            price.append(150)
        elif 15000 < df['ACC33'][i] <= 20000:
            price.append(180)
        else:
            price.append(None)
            error_index.append(i)
    else:
        price.append(None)
        error_index.append(i)

In [152]:
print(len(price))
print(len(error_index))

10104895
1090760


In [153]:
df2 = pd.concat([df, pd.DataFrame(price, columns=['price'])], axis=1)

In [156]:
df2.head(2)

,ACC01,ACC02,ACC03,ACC04,ACC05,ACC06,ACC07,ACC10,ACC11,ACC12,ACC13,ACC14,ACC15,ACC16,ACC17,ACC18,ACC19,ACC20,ACC21,ACC22,ACC23,ACC24,ACC25,ACC26,ACC27,ACC28,ACC29,ACC30,ACC31,ACC32,ACC33,ACC34,ACC35,ACC36,ACC37,ACC38,ACC39,ACC40,局名,price
0,100117,0,1,1,0.0,1070102,75412,100999,NaN,0,1,1,1,4.0,0,0,0,2,0,NaN,0,1,0,0,0,2.0,普通掛號公件信函,NaN,1.0,1,2000,1,90021210011701,90021210011701,NaN,NaN,10701,1,臺北大安,180.0
1,100117,0,1,1,0.0,1070102,83014,100999,NaN,0,1,3,1,0.0,0,0,0,1,0,NaN,0,1,0,0,0,2.0,同縣市快捷,238.0,1.0,1,45,1,93433810011750238006,93433810011750238006,0.0,NaN,10701,1,臺北大安,80.0


In [157]:
# df2.to_csv('/Volumes/transcend/大檔案/中華郵政/雙北市ACC價錢.csv', index=False)

In [167]:
df3 = df2.groupby(['ACC01','局名'])['price'].sum().reset_index()
df4 = df3.sort_values('price', ascending=False).reset_index(drop=True)
df4.head()

,ACC01,局名,price
0,100148,內湖,8498764.0
1,100108,臺北敦南,7683105.0
2,220000,板橋文化路,6472588.0
3,220071,中和,5963271.0
4,100112,臺北興安,5840367.0


In [168]:
df4.to_excel('/Volumes/transcend/大檔案/中華郵政/郵局獲利能力.xlsx')